In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import json

In [3]:
import tfnet

In [4]:
X_train, y_train, X_valid, y_valid, X_test, y_test = tfnet.load_original_data('../traffic-signs-data')

In [5]:
n_classes = len(np.unique(y_train))

In [6]:
channel_means = tfnet.get_image_data_mean_per_channel(X_train)

X_train_scaled = tfnet.image_data_scaling(X_train, channel_means)
X_test_scaled = tfnet.image_data_scaling(X_test, channel_means)
X_valid_scaled = tfnet.image_data_scaling(X_valid, channel_means)

In [7]:
SESSION_CHECKPOINT = 'grid-search-2/nn_2018-01-24_144003'

In [8]:
hyper = json.load(open(SESSION_CHECKPOINT + '_hyper.json'))
hyper

{'conv_layers': [[28, 28, 6], [10, 10, 16]],
 'fc_layers': [120, 84, 43],
 'mean': 0.0,
 'stddev': 0.1}

In [9]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)

convnet_func = tfnet.create_convnet(**hyper)
tensors = convnet_func(x)
y_hat = tensors[-1]
dropout_prob = tensors[0]

training_op = tfnet.create_training_tensor(y_hat, one_hot_y, rate=0.001)
accuracy_op = tfnet.create_accuracy_tensor(y_hat, one_hot_y)

In [10]:
saver = tf.train.Saver()

with tf.Session() as session:
    
    session.run(tf.global_variables_initializer())    
    accuracy_1 = tfnet.evaluate_accuracy(accuracy_op, x, y, dropout_prob, X_valid, y_valid, batch_size=128)
    print(accuracy_1)
    
    saver.restore(session, SESSION_CHECKPOINT)
    accuracy_2 = tfnet.evaluate_accuracy(accuracy_op, x, y, dropout_prob, X_valid_scaled, y_valid, batch_size=128)
    print(accuracy_2)


0.0226757369615
0.956462585034
